## Doc2Vec

このノートブックでは、gensimを用いて、Doc2vecのモデルを学習する方法を紹介します。

### パッケージのインポート

gensimとNLTKはインストール済みとします。

In [1]:
import warnings
from pprint import pprint

import nltk
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
nltk.download('punkt')
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


### 学習データの準備

まずは、gensimでDoc2Vecを学習するためのデータとして、[TaggedDocument](https://radimrehurek.com/gensim/models/doc2vec.html#gensim.models.doc2vec.TaggedDocument)を用意します。
TaggedDocumentは、単語の系列とタグの系列から構成されます。タグは1つ以上の文字列を使えますが、メモリ効率の観点から一意な整数のIDを使うことがほとんどです。



In [2]:
data = [
    "dog bites man",
    "man bites dog",
    "dog eats meat",
    "man eats food"
]

tagged_data = [
    TaggedDocument(
        words=word_tokenize(word),
        tags=[str(i)]
    ) for i, word in enumerate(data)
]

In [3]:
tagged_data

[TaggedDocument(words=['dog', 'bites', 'man'], tags=['0']),
 TaggedDocument(words=['man', 'bites', 'dog'], tags=['1']),
 TaggedDocument(words=['dog', 'eats', 'meat'], tags=['2']),
 TaggedDocument(words=['man', 'eats', 'food'], tags=['3'])]

### モデルの学習

学習データを用意したら、[Doc2Vec](https://radimrehurek.com/gensim/models/doc2vec.html#gensim.models.doc2vec.Doc2Vec)クラスに与えて、モデルを学習します。
指定できるパラメータは様々ありますが、`dm`で学習アルゴリズムを指定できます。1を指定すると、分散メモリ、0を指定すると分散BoWになります。



In [4]:
# 分散BoWの学習
model_dbow = Doc2Vec(
    tagged_data,
    vector_size=20,
    min_count=1,
    epochs=2,
    dm=0
)

学習を終えたら、モデルの[inifer_vector](https://radimrehurek.com/gensim/models/doc2vec.html#gensim.models.doc2vec.Doc2Vec.infer_vector)メソッドに文書を与えて、ベクトルを取得します。

In [5]:
print(model_dbow.infer_vector(['man', 'eats', 'food']))

[-0.01564034  0.01738331 -0.0051671   0.00376444 -0.01813931 -0.00460707
 -0.01941582 -0.01952396 -0.0067725  -0.00411686  0.00786543  0.01587096
 -0.00982575 -0.02477862  0.00217828  0.02137299 -0.00618656  0.00858943
  0.01089256 -0.01651034]


In [6]:
model_dbow.wv.most_similar("man", topn=5) # top 5 most simlar words.

[('dog', 0.3310743570327759),
 ('eats', 0.2360897958278656),
 ('meat', 0.052991606295108795),
 ('food', -0.0032464265823364258),
 ('bites', -0.41033852100372314)]

`n_similarity`を使って、2つの単語集合間の類似度を計算します。

In [7]:
 model_dbow.wv.n_similarity(["dog"],["man"])

0.33107436

In [8]:
# 分散メモリの学習
model_dm = Doc2Vec(
    tagged_data,
    min_count=1,
    vector_size=20,
    epochs=2,
    dm=1
)

print("Inference Vector of man eats food")
print(model_dm.infer_vector(['man', 'eats', 'food']))

print("Most similar words to man in our corpus")
print(model_dm.wv.most_similar("man",topn=5))

print("Similarity between man and dog: ", model_dm.wv.n_similarity(["dog"],["man"]))

Inference Vector of man eats food
[-0.01564045  0.0173833  -0.00516716  0.0037643  -0.01813941 -0.00460716
 -0.01941588 -0.01952404 -0.00677244 -0.00411688  0.00786548  0.01587102
 -0.00982586 -0.02477862  0.00217828  0.02137304 -0.00618664  0.00858937
  0.01089258 -0.01651028]
Most similar words to man in our corpus
[('dog', 0.3310743570327759), ('eats', 0.2360897958278656), ('meat', 0.052991606295108795), ('food', -0.0032464265823364258), ('bites', -0.41033852100372314)]
Similarity between man and dog:  0.33107436


ボキャブラリに存在しない単語を類似度の比較に使うと何が起きるでしょうか？

In [9]:
model_dm.wv.n_similarity(['covid'],['man'])

KeyError: ignored